In [ ]:
#BUNDARAN HI
INPUT_FILENAME = r"C:\Users\user\OneDrive\IPB\Thesis\02. Development\04. Simulasi Backtrajectory HYSPLIT\20251116\bundaranhi\tdump.175026.txt"
OUTPUT_FILENAME = r"C:\Users\user\OneDrive\IPB\Thesis\02. Development\04. Simulasi Backtrajectory HYSPLIT\20251116\bundaranhi\output\trajectory_data.csv"

#KELAPAGADING
INPUT_FILENAME = r"C:\Users\user\OneDrive\IPB\Thesis\02. Development\04. Simulasi Backtrajectory HYSPLIT\20251116\kelapagading\tdump.175790.txt"
OUTPUT_FILENAME = r"C:\Users\user\OneDrive\IPB\Thesis\02. Development\04. Simulasi Backtrajectory HYSPLIT\20251116\kelapagading\output\trajectory_data.csv"

#JAGAKARSA
INPUT_FILENAME = r"C:\Users\user\OneDrive\IPB\Thesis\02. Development\04. Simulasi Backtrajectory HYSPLIT\20251116\jagakarsa\tdump.172842.txt"
OUTPUT_FILENAME = r"C:\Users\user\OneDrive\IPB\Thesis\02. Development\04. Simulasi Backtrajectory HYSPLIT\20251116\jagakarsa\output\trajectory_data.csv"

#LUBANGBUAYA
INPUT_FILENAME = r"C:\Users\user\OneDrive\IPB\Thesis\02. Development\04. Simulasi Backtrajectory HYSPLIT\20251116\lubangbuaya\tdump.175929.txt"
OUTPUT_FILENAME = r"C:\Users\user\OneDrive\IPB\Thesis\02. Development\04. Simulasi Backtrajectory HYSPLIT\20251116\lubangbuaya\output\trajectory_data.csv"

#KEBUNJERUK
INPUT_FILENAME = r"C:\Users\user\OneDrive\IPB\Thesis\02. Development\04. Simulasi Backtrajectory HYSPLIT\20251116\kebunjeruk\tdump.176118.txt"
OUTPUT_FILENAME = r"C:\Users\user\OneDrive\IPB\Thesis\02. Development\04. Simulasi Backtrajectory HYSPLIT\20251116\kebunjeruk\output\trajectory_data.csv"

In [ ]:
#BUNDARAN HI
INPUT_FILENAME = r"C:\Users\user\OneDrive\IPB\Thesis\02. Development\04. Simulasi Backtrajectory HYSPLIT\20251120\bundaranhi\tdump.128242.txt"
OUTPUT_FILENAME = r"C:\Users\user\OneDrive\IPB\Thesis\02. Development\04. Simulasi Backtrajectory HYSPLIT\20251120\bundaranhi\output\trajectory_data.csv"

#KELAPAGADING
INPUT_FILENAME = r"C:\Users\user\OneDrive\IPB\Thesis\02. Development\04. Simulasi Backtrajectory HYSPLIT\20251120\kelapagading\tdump.128307.txt"
OUTPUT_FILENAME = r"C:\Users\user\OneDrive\IPB\Thesis\02. Development\04. Simulasi Backtrajectory HYSPLIT\20251120\kelapagading\output\trajectory_data.csv"

#JAGAKARSA
INPUT_FILENAME = r"C:\Users\user\OneDrive\IPB\Thesis\02. Development\04. Simulasi Backtrajectory HYSPLIT\20251120\jagakarsa\tdump.128386.txt"
OUTPUT_FILENAME = r"C:\Users\user\OneDrive\IPB\Thesis\02. Development\04. Simulasi Backtrajectory HYSPLIT\20251120\jagakarsa\output\trajectory_data.csv"

#LUBANGBUAYA
INPUT_FILENAME = r"C:\Users\user\OneDrive\IPB\Thesis\02. Development\04. Simulasi Backtrajectory HYSPLIT\20251120\lubangbuaya\tdump.128517.txt"
OUTPUT_FILENAME = r"C:\Users\user\OneDrive\IPB\Thesis\02. Development\04. Simulasi Backtrajectory HYSPLIT\20251120\lubangbuaya\output\trajectory_data.csv"

#KEBUNJERUK
INPUT_FILENAME = r"C:\Users\user\OneDrive\IPB\Thesis\02. Development\04. Simulasi Backtrajectory HYSPLIT\20251120\kebunjeruk\tdump.128656.txt"
OUTPUT_FILENAME = r"C:\Users\user\OneDrive\IPB\Thesis\02. Development\04. Simulasi Backtrajectory HYSPLIT\20251120\kebunjeruk\output\trajectory_data.csv"


In [1]:
import pandas as pd
import re
import numpy as np 

# Nama file input dan output
INPUT_FILENAME = r"C:\Users\user\OneDrive\IPB\Thesis\02. Development\04. Simulasi Backtrajectory HYSPLIT\20251120\bundaranhi\tdump.128242.txt"
OUTPUT_FILENAME = r"C:\Users\user\OneDrive\IPB\Thesis\02. Development\04. Simulasi Backtrajectory HYSPLIT\20251120\bundaranhi\output\trajectory_data.csv"

# --- Definisi Fungsi Pemrosesan ---
def process_trajectory_dump_stable_final(input_file, output_file):
    """
    Memproses file dump trajectory, menggunakan indeks yang aman dan 
    mengoreksi urutan LAT/LON di output CSV.
    """
    
    data_list = []
    
    # DAFTAR KOLOM YANG DIHARAPKAN (urutan yang benar)
    # Ini adalah 12 kolom data yang kita butuhkan dari file mentah (setelah dipecah)
    COLUMNS = [
        'DATETIME', 'LATITUDE', 'LONGITUDE', 'PRESSURE_MB', 'THETA_K', 
        'AIR_TEMP_K', 'RAINFALL_MM', 'MIXDEPTH_M', 'RELHUMID_PERCENT', 
        'TERR_MSL_M', 'SUN_FLUX', 'WIND_U_MS', 'WIND_V_MS'
    ]
    
    try:
        with open(input_file, 'r') as f:
            for line in f:
                line_clean = line.strip()
                
                # Filter robust: hanya memproses baris yang dimulai dengan 1 1, 2 1, atau 3 1
                if not re.match(r'^\s*[1-3]\s+1', line_clean):
                    continue
                
                # Memecah baris berdasarkan spasi dan menghilangkan string kosong
                tokens = [t for t in line_clean.split() if t]
                
                # Memastikan ada cukup token (minimal 12 token setelah token status/waktu)
                # Token [0] sampai [8] adalah waktu/status. Kita butuh minimal 9 token (indeks 9) untuk LAT/LON
                if len(tokens) < 11: # Harus ada minimal LAT dan LON
                    continue
                    
                try:
                    # --- 1. EKSTRAKSI WAKTU ---
                    # Tokens[2] hingga tokens[5]
                    tahun, bulan, hari, jam = map(int, tokens[2:6])
                    datetime_str = f'20{tahun:02d}-{bulan:02d}-{hari:02d} {jam:02d}:00:00'
                    
                    # --- 2. EKSTRAKSI DAN KONVERSI DATA NUMERIK ---
                    
                    # Data Angka dimulai dari token ke-9 (indeks 8). Ini adalah Kolom Awal (0.0, -1.0, dst.)
                    # Kita ambil semua data numerik yang ada, hingga akhir baris
                    
                    numerical_data_raw = []
                    # Mulai dari indeks 8 (Kolom Awal) hingga akhir list tokens
                    for token_index in range(8, len(tokens)): 
                        val = tokens[token_index]
                        try:
                            numerical_data_raw.append(float(val))
                        except ValueError:
                            # Jika konversi gagal (misalnya karena nilai '.' atau kosong), 
                            # ganti dengan NaN (Not a Number)
                            numerical_data_raw.append(np.nan)
                    
                    # Kita memerlukan minimal 3 elemen (Kolom Awal, LAT, LON) untuk memproses
                    if len(numerical_data_raw) < 3: 
                        continue
                        
                    # --- 3. KOREKSI URUTAN DAN PEMBATASAN KOLOM ---
                    
                    # Urutan di 'numerical_data_raw': [Col Awal(0), LAT(1), LON(2), PRESSURE(3), ...]
                    
                    corrected_data = []
                    
                    # Tentukan indeks di numerical_data_raw yang sesuai dengan COLUMNS
                    # Indeks [1] dan [2] adalah LAT/LON (yang kita butuhkan pertama)
                    # Indeks [3] adalah PRESSURE (yang kita butuhkan kedua), dst.
                    # Kita butuh 12 kolom data.
                    
                    # Indeks yang kita inginkan dari numerical_data_raw (indeks list yang diekstrak):
                    # LAT (1), LON (2), PRESSURE (3), THETA (4), AIR_TEMP (5), RAINFALL (6), MIXDEPTH (7), 
                    # RELHUMID (8), TERR_MSL (9), SUN_FLUX (10), WIND_U (11), WIND_V (12)
                    
                    target_indices = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
                    
                    for idx in target_indices:
                        if idx < len(numerical_data_raw):
                            # Ambil data jika indeks tersedia
                            corrected_data.append(numerical_data_raw[idx])
                        else:
                            # Jika kolom data tidak tersedia (baris terpotong), isi dengan NaN
                            corrected_data.append(np.nan)
                    
                    # Baris data yang siap dimasukkan ke DataFrame: DATETIME + corrected_data
                    row = [datetime_str] + corrected_data
                    data_list.append(row)
                    
                except Exception as e:
                    print(f"Melewati baris karena error tak terduga: {e} di baris: {line_clean}")
                    continue

        # Buat DataFrame
        # Data yang di list data_list sudah memiliki panjang yang sama (1 + 12 = 13 kolom)
        df = pd.DataFrame(data_list, columns=COLUMNS)
        
        # Simpan ke file CSV
        df.to_csv(output_file, index=False, na_rep='NaN')
        
        print(f"\n--- HASIL PEMROSESAN ---")
        print(f"✅ Pemrosesan berhasil! Data tersimpan di: {output_file}")
        print(f"Total baris data yang diekstrak: {len(df)}")
        print("\n**5 baris data pertama yang sudah diperbaiki:**")
        print(df[['DATETIME', 'LATITUDE', 'LONGITUDE', 'PRESSURE_MB']].head(5))
        
    except FileNotFoundError:
        print(f"❌ Error: File input '{input_file}' tidak ditemukan.")
    except Exception as e:
        print(f"❌ Terjadi error fatal saat pemrosesan: {e}")

# --- Eksekusi Skrip ---
process_trajectory_dump_stable_final(INPUT_FILENAME, OUTPUT_FILENAME)


--- HASIL PEMROSESAN ---
✅ Pemrosesan berhasil! Data tersimpan di: C:\Users\user\OneDrive\IPB\Thesis\02. Development\04. Simulasi Backtrajectory HYSPLIT\20251120\bundaranhi\output\trajectory_data.csv
Total baris data yang diekstrak: 147

**5 baris data pertama yang sudah diperbaiki:**
              DATETIME  LATITUDE  LONGITUDE  PRESSURE_MB
0  2023-10-01 00:00:00    -6.195    106.824        100.0
1  2023-10-01 00:00:00    -6.195    106.824        500.0
2  2023-10-01 00:00:00    -6.195    106.824       1000.0
3  2023-09-30 23:00:00    -6.220    106.912        109.3
4  2023-09-30 23:00:00    -6.210    106.913        511.1


In [2]:
import pandas as pd
import numpy as np

INPUT_FILE = r"C:\Users\user\OneDrive\IPB\Thesis\02. Development\04. Simulasi Backtrajectory HYSPLIT\20251120\bundaranhi\output\trajectory_data.csv"
output_dir = r"C:\Users\user\OneDrive\IPB\Thesis\02. Development\05. Landcover\20251120\bundaranhi"

# Baca data
df = pd.read_csv(INPUT_FILE)

# Buat group index
group1 = np.arange(0, len(df), 3)  # 0, 3, 6, 9, ...
group2 = np.arange(1, len(df), 3)  # 1, 4, 7, 10, ...  
group3 = np.arange(2, len(df), 3)  # 2, 5, 8, 11, ...

# Pisahkan dan simpan
df.iloc[group1].to_csv(f"{output_dir}/trajectory_group_1.csv", index=False)
df.iloc[group2].to_csv(f"{output_dir}/trajectory_group_2.csv", index=False)
df.iloc[group3].to_csv(f"{output_dir}/trajectory_group_3.csv", index=False)

print("✅ Data berhasil dipisahkan menjadi 3 file!")

✅ Data berhasil dipisahkan menjadi 3 file!
